
<p>For this assignment, use the included data file, <code>fxdata.xls</code>.</p>
<ul>
<li>You are given monthly data.</li>
<li>The first tab gives one-month risk-free rates for the U.S. and six other countries. Let $R^{f,i}$ denote each currency for $i=1,...,6$.</li>
<li>For risk-free rate data, $R^{f,i}_{t,t+1}$, the rate is known and reported in the data at time $t$. Namely, any given date $t$ in the data is reporting both $S^i_t$ and $R^{f,i}_{t,t+1}$.</li>
<li>The second tab gives FX rates $S^i_t$ for the six countries in relation to the U.S.</li>
<li>For each currency, the FX rate $S^i_t$ indicates how many USD per foreign currency.</li>
</ul>


In [1]:

import pandas as pd
import numpy as np
import xarray as xr
#import statsmodels.api as sm
import scipy.stats
#!pip install scipy==1.2  --user
import statsmodels.api as sm
pd.set_option('display.float_format', lambda x: '%.4f' % x)
np.set_printoptions(precision=2, suppress=True)



In [2]:

path = 'fxdata.xls'
df_rf = pd.read_excel(path, sheet_name='risk-free').set_index('date')
df_FX = pd.read_excel(path, sheet_name='FX').set_index('date')




<h1 id="Problem-1">Problem 1<a class="anchor-link" href="#Problem-1">¶</a></h1><p>Define the log return of holding the foreign currency using log values of the risk-free rate and log values of the FX rates:</p>
$$
{\texttt{r}}^i_{t+1} = \log R^i_{t+1} = s^i_{t+1} - s^i_t + \texttt{r}^{f,i}_{t,t+1}.
$$<p>Using this, construct the excess log return relative to the USD,</p>
$$
\tilde {\texttt{r}}^i_{t+1} = \log R^i_{t+1} - \log R^{f, \$}_{t, t+1} = s^i_{t+1} - s^i_t + \texttt{r}^{f,i}_{t,t+1} - \texttt{r}^{f,\$}_{t,t+1},
$$<p>for each foreign currency, where $s^i_t = \log S_t^i$ and
$\texttt{r}^{f,i}_{t,t+1} = \log R^{f,i}_{t,t+1}$. Construct a table with the foreign currencies as the rows and the mean, standard deviation, and Sharpe ratio as the columns. Make sure to annualize them. This table should look like the following:
<img alt="Problem 1 Table" src="table_p1.png"/></p>


In [3]:

df_tilde_r = pd.DataFrame(index=df_FX.index, columns=df_FX.columns)
for country in df_FX.columns:
    df_tilde_r[country] = (
        np.log(df_FX[country])
        - np.log(df_FX[country]).shift(1)
        + np.log(df_rf[country]).shift(1)
        - np.log(df_rf['USD']).shift(1)
        )



In [4]:

table_p1 = pd.DataFrame(index=df_FX.columns,
    columns=['mean', 'std', 'sharpe_ratio'])

table_p1['mean'] = df_tilde_r.mean() * 12
table_p1['std'] = df_tilde_r.std() * np.sqrt(12)
table_p1['sharpe_ratio'] = table_p1['mean'] / table_p1['std']

table_p1



,mean,std,sharpe_ratio
AUD,0.0595,0.1056,0.5630
CAD,0.0301,0.0668,0.4507
CHF,0.0116,0.0915,0.1268
EUR,0.0063,0.0881,0.0714
GBP,0.0040,0.0760,0.0532
JPY,-0.0118,0.0828,-0.1425



<h1 id="Problem-2">Problem 2<a class="anchor-link" href="#Problem-2">¶</a></h1><p>Suppose that ${\texttt{r}}^i_{t+1} \overset{iid}{\sim} 
\mathcal N(\mu_i, \sigma_i^2)$. The cumulative return is just the
product of the single-period returns,</p>
$$
R^i_{t,t+h} = \prod_{k=1}^h R^i_{t+k}.
$$<p>In logs, this is</p>
$$
{\texttt{r}}^i_{t,t+h} = \sum_{k=1}^h \texttt{r}^i_{t+k}
$$<p>Calculate the probability that the return to investing in the foreign currency $i$ is less than the return to investing in the domestic currency (the US riskless security) over the horizon $h$,</p>
$$
Pr\left[ R^i_{t,t+h} &lt; R^{f,\$}_{t,t+h}\right].
$$<p>Do this for each foreign currency and for the horizons $h=1,5,10,15,20$. Report these probabilities in a table with the foreign currencies as the columns and the five different horizons in the rows. For $\mu_i$ and $\sigma_i$, use the sample mean and standard deviations.</p>
<p><em>Hint:</em> If $X \sim \mathcal N(\mu_x, \sigma_x^2)$, then</p>
$$
Pr\left[x &lt; \ell\right] = \Phi_{\mathcal N}(L)
$$<p>with $L = \frac{\ell - \mu_x}{\sigma_x}$ and $\Phi_{\mathcal N}$ is the standard normal CDF.</p>



<p><strong>SOLUTION:</strong></p>
$$R^i_{t,t+h} &lt; R^{f,\$}_{t,t+h}$$<p></p>
<p>is equivalent to
$$ \sum_{k=1}^h\tilde{\texttt{r}}^i_{t+k} &lt; 0.$$</p>
<p>The return on the risk-free securities have volatility zero, since 
$R^{f,\$}_{t,t+1}$ is known at time $t$. So,</p>
$$
\sum_{k=1}^h\tilde{\texttt{r}}^i_{t+k} \sim \mathcal N(h \mu_i, h \sigma_i^2)
$$<p>and</p>
$$
Pr\left[\sum_{k=1}^h\tilde{\texttt{r}}^i_{t+k} &lt; 0 \right]
= \Phi_{\mathcal N}\left( -\sqrt{h} \frac{\tilde \mu_i}{\tilde \sigma_i}\right)
$$


In [5]:

table_p2 = pd.DataFrame(columns=df_FX.columns, index=[1,5,10, 15, 20])
table_p2.index.name = 'horizon'



In [6]:

for h in table_p2.index:
    row = scipy.stats.norm.cdf(
        -np.sqrt(h) * table_p1['mean'] / table_p1['std'])
    table_p2.loc[h, :] = row



In [7]:

table_p2



,AUD,CAD,CHF,EUR,GBP,JPY
horizon,,,,,,
1,0.2867,0.3261,0.4496,0.4715,0.4788,0.5567
5,0.1040,0.1568,0.3884,0.4366,0.4527,0.6250
10,0.0375,0.0770,0.3442,0.4107,0.4332,0.6739
15,0.0146,0.0404,0.3117,0.3910,0.4184,0.7095
20,0.0059,0.0219,0.2854,0.3747,0.4060,0.7381



<h1 id="Problem-3">Problem 3<a class="anchor-link" href="#Problem-3">¶</a></h1><p>Suppose that we want to invest in a combination of these foreign securities and that we want to use mean-variance optimization
to construct a $\tilde {\text{MV}}$ portfolio. Use the excess log returns $\tilde {\texttt{r}}^i_{t+1}$ to construct
the tangency portfolio based on these currency trades. Report the weights of the tangency portfolio. (Note: From the point of view of the USD, the six foreign securities are risky assets.)</p>


In [8]:

def compute_tangency(df_tilde, diagonalize_Sigma=False):
    """Compute tangency portfolio given a set of excess returns.

    Also, for convenience, this returns the associated vector of average
    returns and the variance-covariance matrix.

    Parameters
    ----------
    diagonalize_Sigma: bool
        When `True`, set the off diagonal elements of the variance-covariance
        matrix to zero.
    """
    Sigma = df_tilde.cov()
    # N is the number of assets
    N = Sigma.shape[0]
    Sigma_adj = Sigma.copy()
    if diagonalize_Sigma:
        Sigma_adj.loc[:,:] = np.diag(np.diag(Sigma_adj))

    mu_tilde = df_tilde.mean()
    Sigma_inv = np.linalg.inv(Sigma_adj)
    weights = Sigma_inv @ mu_tilde / (np.ones(N) @ Sigma_inv @ mu_tilde)
    # For convenience, I'll wrap the solution back into a pandas.Series object.
    omega_tangency = pd.Series(weights, index=mu_tilde.index)
    return omega_tangency, mu_tilde, Sigma



In [9]:

omega_tangency, mu_tilde, Sigma = compute_tangency(df_tilde_r)
omega_tangency



AUD    8.0422
CAD    1.2552
CHF    6.5749
EUR   -9.0334
GBP   -3.7651
JPY   -2.0739
dtype: float64


<h1 id="Problem-4">Problem 4<a class="anchor-link" href="#Problem-4">¶</a></h1><p>Now, recompute the tangency portfolio from before. This time, diagonalize the variance-covariance matrix so that the covariances between the risky assets are all zero. Report the new tangency portfolio. How does it change?</p>


In [10]:

omega_tangency, mu_tilde, Sigma = compute_tangency(df_tilde_r, 
                                                   diagonalize_Sigma=True)
omega_tangency # The positions are much less extreme.



AUD    0.4022
CAD    0.5092
CHF    0.1046
EUR    0.0612
GBP    0.0528
JPY   -0.1299
dtype: float64


<h1 id="Problem-5">Problem 5<a class="anchor-link" href="#Problem-5">¶</a></h1><p>For each foreign currency, compute $\alpha^i$ and $\beta^i$ for each of the regressions</p>
$$
s_{t+1}^i - s_t^i = \alpha^i + \beta^i \left(\texttt{r}_{t,t+1}^{f,\$}
 - \texttt{r}_{t,t+1}^{f,i}\right) + \epsilon_{t+1}^i,
$$<p>where $\texttt{r}^{f,i}$ denotes the log risk-free rate of currency $i$
and $s^i$ denotes the FX rate for currency $i$. Again, note that both
$\texttt{r}^{f,\$}_{t,t+1}$ and $s_t$ are determined at time $t$.
Make a table with 6 rows. Each row is a different currency. In the columns, report $\alpha^i$, $\beta^i$, and the r-squared of the regression. Your results should match the table reported below.</p>
<p><img alt="Table, Problem 2" src="table_p_1_2.png"/></p>


In [11]:

df_regressions = pd.DataFrame(index=['const', 'x1', 'r_squared'],
    columns=df_FX.columns)
for country in df_regressions.columns:
    lhs = np.log(df_FX[country]) - np.log(df_FX[country]).shift(1)
    differential = np.log(df_rf['USD']).shift(1) - np.log(df_rf[country]).shift(1)
    rhs = sm.add_constant(differential.values)
    res = sm.OLS(lhs, rhs, missing='drop').fit()
    df_regressions.loc[['const', 'x1'], country] = res.params
    df_regressions.loc[['r_squared'], country] = res.rsquared

    last_date = differential.index[-1]
    exog = [1, differential[last_date]]
df_regressions = df_regressions.rename(index={'x1': 'beta'})

df_regressions.T



,const,beta,r_squared
AUD,0.0018,-0.5584,0.0007
CAD,0.0025,1.1001,0.0016
CHF,0.0054,-2.4879,0.0134
EUR,0.0009,-1.8950,0.0077
GBP,0.0005,1.1953,0.0027
JPY,0.0022,-0.5440,0.0016



<h1 id="Problem-6">Problem 6<a class="anchor-link" href="#Problem-6">¶</a></h1><p>Recall that in the previous homework, we used the estimates from the previous regression to
construct a time $t$ forecast of the excess log holding period return on each foreign currency .
Specifically, we did this by using the estimated $\alpha^i$ and $\beta^i$ to compute</p>
$$
\mathbb E_t[\tilde{\texttt{r}}^i_{t+1}] = \alpha^i + (\beta^i-1)
\left(\texttt{r}_{t,t+1}^{f,\$} - \texttt{r}_{t,t+1}^{f,i}\right).
$$<p>We could use this information to construct a dynamic trading strategy. However,
this procudure has a problem that prevents us from evaluating its true 
usefulness in practice. 
Namely, the $\alpha^i$ and $\beta^i$ used to construct our predictions
$ \mathbb E_t[\tilde{\texttt{r}}^i_{t+1}]$
are estimated using data from dates later than time $t$.</p>
<p>In this exercise, construct the time series for $ \mathbb E_t[\tilde{\texttt{r}}^i_{t+1}]$, this time using estimates $\alpha^i_t$ and $\beta^i_t$, where $\alpha^i_t$ and $\beta^i_t$ are estimating using data available only up to time $t$. 
Note that each regression should have, at a minimum, 2 or more data points. Use an expanding window so that, with each time period, the amount of data to feed into the regression increases.</p>
<p>For each currency, compute how often $ \mathbb E_t[\tilde{\texttt{r}}^i_{t+1}] &gt; 0$. Report the percentage of times that this is true for each currency.</p>


In [12]:

regs = xr.DataArray(
    dims=('currency', 'params', 'date'),
    coords={
        'currency': df_FX.columns,
        'params': ['const', 'x1'],
        'date': df_FX.index
    }
    )

for it, t in enumerate(df_FX.index):
    if it < 1:
        pass
    else:
        for icountry, country in enumerate(df_FX.columns):
            lhs = np.log(df_FX.loc[:t, country]) - np.log(df_FX.loc[:t, country]).shift(1)
            differential = np.log(df_rf.loc[:t, 'USD']).shift(1) - np.log(df_rf.loc[:t, country]).shift(1)
            rhs = sm.add_constant(differential.values)
            res = sm.OLS(lhs, rhs, missing='drop').fit()
            regs.sel(currency=country, date=t)[...] = res.params



In [13]:

df_differential = -df_rf.loc[:, 'AUD':'JPY'].sub(df_rf.loc[:, 'USD'], axis=0)
cond_tilde_r = (regs.sel(params='const').T + 
                   df_differential * (regs.sel(params='x1') - 1)
                  )



In [14]:

df_cond_tilde_r_back = cond_tilde_r.to_dataframe(name='cee_ret')
df_cond_tilde_r_back = df_cond_tilde_r_back['cee_ret'].unstack()



In [15]:

df_cond_tilde_r_back.head()



currency,AUD,CAD,CHF,EUR,GBP,JPY
date,,,,,,
1999-01-01,nan,nan,nan,nan,nan,nan
1999-02-01,0.0123,0.0145,-0.0327,-0.0356,-0.0127,-0.0332
1999-03-01,-0.0129,-0.0101,-0.0277,-0.0561,0.0180,-0.0397
1999-04-01,-0.0006,0.0196,-0.0288,0.0532,-0.0042,0.0005
1999-05-01,0.0128,0.0206,-0.0430,0.0430,0.0007,-0.0145


In [16]:

(df_cond_tilde_r_back > 0).sum() / len(df_cond_tilde_r_back)



currency
AUD   0.7895
CAD   0.7018
CHF   0.6491
EUR   0.6725
GBP   0.7135
JPY   0.6257
dtype: float64


<h1 id="Problem-7">Problem 7<a class="anchor-link" href="#Problem-7">¶</a></h1><p>We are now ready to incorporate this prediction into a trading strategy
that we can properly back-test.
In this trading strategy, if $ \mathbb E_t[\tilde{\texttt{r}}^i_{t+1}] &gt; 0$, 
we invest 100\% in the foreign currency's riskless asset. 
Otherwise, we invest wholly in the US riskless asset. 
Note that returns should be expressed as log returns.</p>
<p>Compute the mean, standard deviation, and Sharpe ratio of the log returns to this strategy for each foreign currency. Construct a table with the foreign currencies as the rows and the mean, standard deviation, and Sharpe ratio as the columns.</p>


In [17]:

cond = df_tilde_r * (df_cond_tilde_r_back > 0)
df_dynamic_back = cond.add(np.log(df_rf['USD']), axis=0)



In [18]:

table_p8 = pd.DataFrame(index=df_FX.columns,
    columns=['mean', 'std', 'sharpe_ratio'])

table_p8['mean'] = df_dynamic_back.mean() * 12
table_p8['std'] = df_dynamic_back.std() * np.sqrt(12)
table_p8['sharpe_ratio'] = table_p8['mean'] / table_p8['std']

table_p8



,mean,std,sharpe_ratio
AUD,0.1063,0.0944,1.1266
CAD,0.0610,0.0623,0.9795
CHF,0.0623,0.0776,0.8027
EUR,0.0527,0.0747,0.7058
GBP,0.0395,0.0683,0.5782
JPY,0.0502,0.0640,0.7847
